In [1]:
import pandas as pd


In [11]:
# Estimativa de pedidos:

# Recebe os arquivos
estimativa = pd.read_excel("")

# Remove colunas desnecessárias
estimativa.drop(['Data Pedido','Conta','Endereço de Coleta', 'Envio Recebido',
            'Saiu para entrega', 'Primeira Visita', 'Entrega',
            'Endereço', "Complemento", 'Bairro', 'Latitude', 'Longitude',
            'Comprador', 'Id Envio', 'Id Pedido', 'Simulação valor do frete',
            'Id Usuário Mercado Livre', 'Telefone contato comprador',
            'CEP Vendedor', 'Preferência de entrega', 'Código Cliente',
            'Total Produtos R$', 'Tamanho', 'Dimensões do(s) Produto(s)',
            'Nome do(s) Produto(s)', 'Zona de entrega', 'Data de cancelamento',
            'Nome Motorista'], axis=1,inplace=True)

# Cria a coluna de C-CEP
estimativa['C-CEP'] = estimativa['CEP'].astype(str).str[:3]

# Remove os pedidos que contem status na Log
estimativa.drop(estimativa[~(estimativa['Status TMS'].isna() |
 (estimativa['Status TMS'] == ""))].index, inplace=True)

# Remove os pedidos que estejam entregues ou cancelado no canal
estimativa.drop(estimativa[estimativa['Status Canal'].isin(['ENTREGUE', 'CANCELADO'])].index, inplace=True)

# Dicionário de mapeamento: região -> lista de C-CEP
regioes = {
    'CENTRO':   ['010', '011', '012', '013', '014', '015'],
    'NORTE':    ['020', '021', '022', '023', '024', '025', '026', '027', '028', '029'],
    'LESTE 1':  ['030', '031', '032', '033', '034', '035', '036', '082'],
    'LESTE 2':  ['037', '038', '039', '080', '083'],
    'SUL':      ['040', '041', '042', '043', '044', '045', '046', '047', '048'],
    'OESTE':    ['050', '051', '053', '054', '055', '056', '057', '067', '068'],
    'EXT. OESTE': ['060', '061', '062', '063', '064', '065', '066', '077', '078', '079'],
    'GUARULHOS': ['081', '084', '085', '086', '087', '088', '070', '071', '072', '074', '076'],
    'ABC':      ['090', '091', '092', '093', '094', '095', '096', '097', '098', '099']
}

def identificar_regiao(ccep):
    for regiao, lista in regioes.items():
        if ccep in lista:
            return regiao
    return 'DESCONHECIDO'  # caso não caia em nenhuma regra

# Criando a nova coluna 'Região'
estimativa['Região'] = estimativa['C-CEP'].apply(identificar_regiao)

# Ajuste de C-CEP para cidades específicas
ajustes_cidades = {
    'Itaquaquecetuba': ('085', 'Itaqua'),
    'Cajamar': ('077', 'Cajamar'),
    'Taboão da Serra': ('067', 'Taboão'),
    'Cotia': ('067', 'Cotia'),
    'Ferraz de Vasconcelos': ('085', 'Ferraz'),
    'Itapevi': ('066', 'Itapevi'),
    'Jandira': ('066', 'Jandira'),
    'Poa': ('085', 'Ferraz'),
    'Poá': ('085', 'Ferraz'),
    'Caieiras': ('077', 'Caieiras')
}

def ajustar_ccep(row):
    cidade = row['Cidade']  # Certifique-se de que a coluna se chama "Cidade"
    ccep_original = row['C-CEP']

    if cidade in ajustes_cidades:
        prefixo, nome = ajustes_cidades[cidade]
        # Mantém o prefixo original, apenas coloca o nome da cidade
        return f"{prefixo} - {nome}"
    return ccep_original

estimativa['C-CEP'] = estimativa.apply(ajustar_ccep, axis=1)

# Criação da coluna Canal
estimativa['Canal'] = estimativa['Id Etiqueta'].astype(str).str[:3].apply(
    lambda x: 'Shopee' if x in ['BR2', '251'] else 'Mercado Livre'
)

# Elimando colunas que não serão mais utilizadas
estimativa.drop(['Cidade','Status Canal', 'Status TMS', 'CEP'],
                axis=1, inplace=True)

# Reordenando as colunas
colunas_ordenadas = ['Id Etiqueta', 'Canal', 'Cliente', 'C-CEP', 'Região']

# Aplica a nova ordem
estimativa = estimativa[colunas_ordenadas]

# Exporta pra um arquivo .xlsx
#estimativa.to_excel('ESTIMATIVA DE PEDIDOS 17-11 (9h).xlsx', index=False)

In [ ]:
# Recebimento Físico

# Recebe os arquivos
recebimento = pd.read_excel("")

# Remove colunas desnecessárias
recebimento.drop(['Data Pedido','Conta','Endereço de Coleta', 'Envio Recebido',
            'Saiu para entrega', 'Primeira Visita', 'Entrega',
            'Endereço', "Complemento", 'Bairro', 'Latitude', 'Longitude',
            'Comprador', 'Id Envio', 'Id Pedido', 'Simulação valor do frete',
            'Id Usuário Mercado Livre', 'Telefone contato comprador',
            'CEP Vendedor', 'Preferência de entrega', 'Código Cliente',
            'Total Produtos R$', 'Tamanho', 'Dimensões do(s) Produto(s)',
            'Nome do(s) Produto(s)', 'Zona de entrega', 'Data de cancelamento',
            'Nome Motorista'], axis=1,inplace=True)

# Cria a coluna de C-CEP
recebimento['C-CEP'] = recebimento['CEP'].astype(str).str[:3]

# Remove linhas onde Status TMS está vazio
recebimento.drop(recebimento[recebimento['Status TMS'].isna() |
                           (recebimento['Status TMS'] == "")].index, inplace=True)


# Dicionário de mapeamento: região -> lista de C-CEP
regioes = {
    'CENTRO':   ['010', '011', '012', '013', '014', '015'],
    'NORTE':    ['020', '021', '022', '023', '024', '025', '026', '027', '028', '029'],
    'LESTE 1':  ['030', '031', '032', '033', '034', '035', '036', '082'],
    'LESTE 2':  ['037', '038', '039', '080', '083'],
    'SUL':      ['040', '041', '042', '043', '044', '045', '046', '047', '048'],
    'OESTE':    ['050', '051', '053', '054', '055', '056', '057', '067', '068'],
    'EXT. OESTE': ['060', '061', '062', '063', '064', '065', '066', '077', '078', '079'],
    'GUARULHOS': ['081', '084', '085', '086', '087', '088', '070', '071', '072', '074', '076'],
    'ABC':      ['090', '091', '092', '093', '094', '095', '096', '097', '098', '099']
}

def identificar_regiao(ccep):
    for regiao, lista in regioes.items():
        if ccep in lista:
            return regiao
    return 'DESCONHECIDO'  # caso não caia em nenhuma regra

# Criando a nova coluna 'Região'
recebimento['Região'] = recebimento['C-CEP'].apply(identificar_regiao)

# Ajuste de C-CEP para cidades específicas
ajustes_cidades = {
    'Itaquaquecetuba': ('085', 'Itaqua'),
    'Cajamar': ('077', 'Cajamar'),
    'Taboão da Serra': ('067', 'Taboão'),
    'Cotia': ('067', 'Cotia'),
    'Ferraz de Vasconcelos': ('085', 'Ferraz'),
    'Itapevi': ('066', 'Itapevi'),
    'Jandira': ('066', 'Jandira'),
    'Poa': ('085', 'Ferraz'),
    'Poá': ('085', 'Ferraz'),
    'Caieiras': ('077', 'Caieiras')
}

def ajustar_ccep(row):
    cidade = row['Cidade']  # Certifique-se de que a coluna se chama "Cidade"
    ccep_original = row['C-CEP']

    if cidade in ajustes_cidades:
        prefixo, nome = ajustes_cidades[cidade]
        # Mantém o prefixo original, apenas coloca o nome da cidade
        return f"{prefixo} - {nome}"
    return ccep_original

recebimento['C-CEP'] = recebimento.apply(ajustar_ccep, axis=1)

# Criação da coluna Canal
recebimento['Canal'] = recebimento['Id Etiqueta'].astype(str).str[:3].apply(
    lambda x: 'Shopee' if x in ['BR2', '251'] else 'Mercado Livre'
)

# Elimando colunas que não serão mais utilizadas
recebimento.drop(['Cidade','Status Canal', 'Status TMS', 'CEP'],
                axis=1, inplace=True)

# Reordenando as colunas
colunas_ordenadas = ['Id Etiqueta', 'Canal', 'Cliente', 'C-CEP', 'Região']

# Aplica a nova ordem
recebimento = recebimento[colunas_ordenadas]

# Exporta pra um arquivo .xlsx
#recebimento.to_excel('PEDIDOS RECEBIDOS 17-11.xlsx', index=False)